Code provided by Library of Congress Chronicling American API guide found here:
https://libraryofcongress.github.io/data-exploration/loc.gov%20JSON%20API/Chronicling_America/ChronAm_analyzing_specific_titles_limit_results.html

In [1]:
import time
import re
import json
from urllib.request import urlopen
import requests
import pandas as pd
import pprint

In [3]:
# Test to get all search results:
searchURLTest = "https://www.loc.gov/collections/chronicling-america/?end_date=1930-12-31&ops=~10!PHRASE&qs=Los+Angeles+oil!Los+Angeles&searchType=advanced&sp=1&start_date=1890-01-01&fo=json"
numberOfResults = 0

def get_item_ids_test(url, items=[], conditional='True'):
    global numberOfResults
    # Check that the query URL is not an item or resource link.
    exclude = ["loc.gov/item","loc.gov/resource"]
    if any(string in url for string in exclude):
        raise NameError('Your URL points directly to an item or '
                        'resource page (you can tell because "item" '
                        'or "resource" is in the URL). Please use '
                        'a search URL instead. For example, instead '
                        'of \"https://www.loc.gov/item/2009581123/\", '
                        'try \"https://www.loc.gov/maps/?q=2009581123\". ')

    # request pages of 100 results at a time
    params = {"fo": "json", "c": 100, "at": "results,pagination"}
    call = requests.get(url, params=params)
    # Check that the API request was successful
    if (call.status_code==200) & ('json' in call.headers.get('content-type')):
        data = call.json()
        results = data['results'] # deleted the top 20 limit
        for result in results:
            # Filter out anything that's a collection or web page
            filter_out = ("collection" in result.get("original_format")) \
                    or ("web page" in result.get("original_format")) \
                    or (eval(conditional)==False)
            if not filter_out:
                # Get the link to the item record
                if result.get("id"):
                    item = result.get("id")
                    # Filter out links to Catalog or other platforms
                    if item.startswith("http://www.loc.gov/resource"):
                      resource = item  # Assign item to resource
                      items.append(resource)
                    if item.startswith("http://www.loc.gov/item"):
                      items.append(item)
                numberOfResults += 1
        # Repeat the loop on the next page, unless we're on the last page.
        if data["pagination"]["next"] is not None:
            next_url = data["pagination"]["next"]
            print(f"Total number of pages: {data['pagination']['total']}")
            get_item_ids_test(next_url, items, conditional)

        return items
    else:
            print('There was a problem. Try running the cell again, or check your searchURL.')

# Generate a list of records found from performing a query and save these Item IDs. (Create ids_list based on items found in the searchURL result)
ids_list_test = get_item_ids_test(searchURLTest, items=[])

# Add 'fo=json' to the end of each row in ids_list (All individual ids from from the ids_list are now listed in JSON format in new_ids)
ids_list_json_test = []
for id in ids_list_test:
  if not id.endswith('&fo=json'):
    id += '&fo=json'
  ids_list_json_test.append(id)
ids = ids_list_json_test

print('\nSuccess! Your API Search Query found '+str(len(ids_list_json_test))+' related newspaper pages. Proceed to the next step')

Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
Total number of pages: 292
T

In [5]:
# Create a list of dictionaries to store the item metadata
item_metadata_list = []

counter = 0
# Iterate over the list of item IDs
for item_id in ids_list_json_test:
  item_response = requests.get(item_id)

  # Check if the API call was successful and Parse the JSON response
  if item_response.status_code == 200:
    # Iterate over the ids_list_json list and extract the relevant metadata from each dictionary.
    item_data = item_response.json()
    if 'location_city' not in item_data['item']:
      continue

    # Extract the relevant item metadata
    Newspaper_Title = item_data['item']['newspaper_title']
    Issue_Date = item_data['item']['date']
    Page = item_data['pagination']['current']
    State = item_data['item']['location_state']
    City = item_data['item']['location_city']
    LCCN = item_data['item']['number_lccn']
    Contributor = item_data['item']['contributor_names']
    Batch = item_data['item']['batch']
    pdf = item_data['resource']['pdf']

    # Add the item metadata to the list
    item_metadata_list.append({
        'Newspaper Title': Newspaper_Title,
        'Issue Date': Issue_Date,
        'Page Number': Page,
        'LCCN': LCCN,
        'City': City,
        'State': State,
        'Contributor': Contributor,
        'Batch': Batch,
        'PDF Link': pdf,
    })
    counter += 1
    print(f"Processed {counter} results.")
    

# Change date format to MM-DD-YYYY
for item in item_metadata_list:
  item['Issue Date'] = pd.to_datetime(item['Issue Date']).strftime('%m-%d-%Y')

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(item_metadata_list)

print('\nSuccess! Ready to proceed to the next step!')

Processed 1 results.
Processed 2 results.
Processed 3 results.
Processed 4 results.
Processed 5 results.
Processed 6 results.
Processed 7 results.
Processed 8 results.
Processed 9 results.
Processed 10 results.
Processed 11 results.
Processed 12 results.
Processed 13 results.
Processed 14 results.
Processed 15 results.
Processed 16 results.
Processed 17 results.
Processed 18 results.
Processed 19 results.
Processed 20 results.
Processed 21 results.
Processed 22 results.
Processed 23 results.
Processed 24 results.
Processed 25 results.
Processed 26 results.
Processed 27 results.
Processed 28 results.
Processed 29 results.
Processed 30 results.
Processed 31 results.
Processed 32 results.
Processed 33 results.
Processed 34 results.
Processed 35 results.
Processed 36 results.
Processed 37 results.
Processed 38 results.
Processed 39 results.
Processed 40 results.
Processed 41 results.
Processed 42 results.
Processed 43 results.
Processed 44 results.
Processed 45 results.
Processed 46 result

KeyboardInterrupt: 